In [1]:
# Importing Required modules
import pandas as pd
import mysql.connector as msql
import math
import os
import datetime

In [2]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
#Reading She for STEM Incubator file present on source files
directory_path =(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Source File\02_Source_Kalpana incubator WN")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    data = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from Kalpana Incubator Sept- Dec 2023 Batch-1704361632176.csv:


In [4]:
data.head()

,Created On,Last Login,Course Name,Name,Email,Segment,Credits,Assigned via Package,Mobile,"What is your Date of Birth (Strictly for Internal Purposes Only, used to match fellows with internships, jobs, experienced jobs etc.)",Name of Parent (Tax Paying),PAN Number (Tax paying Parent),Name of College,Enroll Date,Valid Till,Assigned Through,Start Date,Time Spent(mins),Progress,Kalpana She For STEM Program Calendar - 651168ffe4b02d6243ee8446,Live Session: Kalpana She For STEM Oreintation - 650864d2e4b01f05a724f7b8,"SUK + : Kalpana Inauguration, at 6:30 PM. on Saturday, 30th Sept 2023 - 65086523e4b0a106c0f6312b",Week 0 : How to use Vigyanshaala App/Website - 0oor3hpvud,VigyanShaala YouTube playlist - 650e844ae4b090169718e7fa,"LIVE Ask Me Anything Session - Tuesday, Oct 3, 18:30 PM - 65086dabe4b0c01a1bbf3c44","Master Class ! : Career Exploration - Saturday, Oct 7, 6:30 PM - 652153e8e4b09e4247b99db1",Week 1 : Overview: Program Kalpana - 7zuegm2weg,Video ! : Why women in STEM. What has Kalpana program achieved until now. - 62c7d78d0cf273488220b8d4,Video @ : What we expect from our Kalpana fellows - 62c7d10e0cf273488220b5bf,"LIVE Ask Me Anything Session - Tuesday, Oct 10, 6:30 PM - 65086f10e4b0e873fb9700f2","Workshop : Career Exploration, Friday, Oct 13, 6:30 PM - 65086feae4b08f7d48cdf5a7","Master Class @ : Planning your Applications – Higher Education, Saturday, Oct 14, 6:30 PM - 650870f7e4b055d84f1698b5","Assignment: Career Exploration - Deadline Thursday, Oct 19, 2023 - 6522d14de4b0b9e8e0c8eb1c","Assignment: Planning your Applications – Higher Education - Deadline Thursday, Nov 23, 2023 - 62ecca6b0cf2e02ac087a6e0",Week 2 : Setting Aspirational Goals & SMART goals - rp0p2e1obv,Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119,Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628,Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932,Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946,"LIVE Ask Me Anything Session - Tuesday, Oct 17, 18:30 PM - 650871f9e4b0af5edfac05ec","Workshop : Setting Aspirational Goals & SMART goals, Friday, Oct 20, 6:30 PM - 6508730ae4b0c01a1bbf3e75","Master Class # : Searching and Securing Internships - Saturday, Oct 21, 6:30 PM - 650873c4e4b0412df5285380","Assignment: Goal Setting - Deadline Thursday, Oct 26, 2023 - 652cfbd1e4b028998831a608","Assignment: SMART goal - Deadline Thursday, Oct 26, 2023 - 62a6e2550cf2534aef28d981","Assignment: Searching & Securing Internship - Deadline Thursday, Nov 9, 2023 - 6533b89ce4b0039b28c3762b",Week 3 : Strategic personal planning tool - SWOT analysis - wmsvo0qx5y,Video & : What is SWOT & its importance - 62b2a68c0cf2aa41107c5324,Video * : How to do SWOT analysis - 62b2abc80cf26c3ef038825c,"Video ? : What are Strengths, Weakness, Opportunities, Threats - 62b2aa8d0cf26c3ef0388187",Video !+ : Deep dive into SWOT template with example - 62c56bfa0cf2d3022e87e6c5,"Workshop : SWOT, Friday, Oct 27, 6:30 PM - 6508759de4b045b87ab3f711","Master Class $ : Acing your CV and Resume - Saturday, Oct 28, 6:30 PM - 6508762de4b056fb2f2602a9","Assignment: SWOT - Deadline Thursday, Nov 2, 2023 - 62a6daaa0cf2534aef28d318",Quiz: Strategic planning tools for STEM (SWOT) - 629f26850cf25d70d8d6de57,"Assignment: Acing your CV and Resume - Deadline Thursday, Nov 16, 2023 - 62fb410fe4b09158723693af",Week 4 : RIASEC & IDP - Goal A and Goal B - ux4nan7s3o,Video !! : RIASEC personality - Who you are - 62b2afb70cf215095ac618cd,Video !@ : Why IDP and it's importance - 62b3f8d60cf2aa41107cb2f2,Video !# : Brief Introduction - 653cc3bae4b084f5408cab20,Video !$ : Setting aspirational goals - 653cc3cee4b0be3f4a4b8223,Video !% : Avoiding 3 common dangers in goal setting - 653cc4dde4b016a1c3877568,Video !^ : Overview - CAP template - 653cc4b2e4b05c1a6befe748,Video !& : Goal A and Goal B - 653cc4c1e4b0be3f4a4b8333,"LIVE Ask Me Anything Session - Tuesday, Oct 31, 18:30 PM - 65087773e4b0a0d7343ddf83","Workshop : RIASEC & IDP - Goal A and Goal B, Friday, Nov 

In [5]:
# Define a list of column names to extract for putting into database 
columns_to_extract = ['Name', 'Email', 'Segment', 'Mobile', 'Enroll Date', 'Assigned Through', 'Course Name']

# Use the loc method to extract the specified columns
df = data.loc[:, columns_to_extract]


In [6]:
# Create a new dataframe with 'email' column and columns that start with 'Week' 'Video', 'Recording', and 'Master class'
data = data[['Email'] + [col for col in data.columns if col.startswith(('Week','Video', 'SUK', 'Master Class'))]]


In [7]:
oldcol = list(data.columns)


In [8]:
#oldcol =data.columns

In [9]:
oldcol

['Email',
 'SUK + : Kalpana Inauguration, at 6:30 PM. on Saturday, 30th Sept 2023 - 65086523e4b0a106c0f6312b',
 'Week 0 : How to use Vigyanshaala App/Website - 0oor3hpvud',
 'Master Class ! : Career Exploration - Saturday, Oct 7, 6:30 PM - 652153e8e4b09e4247b99db1',
 'Week 1 : Overview: Program Kalpana - 7zuegm2weg',
 'Video ! : Why women in STEM. What has Kalpana program achieved until now. - 62c7d78d0cf273488220b8d4',
 'Video @ : What we expect from our Kalpana fellows - 62c7d10e0cf273488220b5bf',
 'Master Class @ : Planning your Applications – Higher Education, Saturday, Oct 14, 6:30 PM - 650870f7e4b055d84f1698b5',
 'Week 2 : Setting Aspirational Goals & SMART goals - rp0p2e1obv',
 'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
 'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
 'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
 'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
 'Master Class # : Se

In [10]:
# Cheaking shape of our dataset
data.shape

(1115, 38)

In [11]:
# Initialize variables to keep track of the current week, video count, recording count, and master class count
week_col = None
video_count = 0
recording_count = 0
master_count = 0

# Create an empty list to store the new column names
new_cols = []

# Iterate over each column in the data
for col in data.columns:
    # If the column starts with 'Week'
    if col.startswith('Week'):
        # Split the column name by space and get the second element (the week number)
        week_col = col.split()[1]
        # Reset the video, recording, and master class counts for the new week
        video_count = 0
        recording_count = 0
    # If the column starts with 'Video'
    elif col.startswith('Video'):
        # Increment the video count for the current week
        video_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_V{video_count}')
    # If the column starts with 'Recording'
    elif col.startswith('SUK'):
        # Increment the recording count for the current week
        recording_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_SUK_V')
    # If the column starts with 'Master Class'
    elif col.startswith('Master Class'):
        # Increment the master class count for the current week
        master_count += 1
        # Append a new column name to the list using f-string formatting
        new_cols.append(f'WK{week_col}_Master{master_count}')
    # If the column doesn't start with any of the above prefixes
    else:
        # Append the original column name to the list
        new_cols.append(col)

# Remove all columns that start with 'Week' from the data
data = data.loc[:, ~data.columns.str.startswith('Week')]
# Assign the new column names to the data
data.columns = new_cols


In [12]:
# Rechecking the shape of dataframe
data.shape

(1115, 29)

In [13]:
# Cheaking columns name
data.columns

Index(['Email', 'WKNone_SUK_V', 'WK0_Master1', 'WK1_V1', 'WK1_V2',
       'WK1_Master2', 'WK2_V1', 'WK2_V2', 'WK2_V3', 'WK2_V4', 'WK2_Master3',
       'WK3_V1', 'WK3_V2', 'WK3_V3', 'WK3_V4', 'WK3_Master4', 'WK4_V1',
       'WK4_V2', 'WK4_V3', 'WK4_V4', 'WK4_V5', 'WK4_V6', 'WK4_V7',
       'WK4_Master5', 'WK5_V1', 'WK6_SUK_V', 'WK7_V1', 'WK8_SUK_V',
       'WK8_SUK_V'],
      dtype='object')

In [14]:
# Check if there are two columns with the name "WK8_SUK_V"
if sum(data.columns == 'WK8_SUK_V') == 2:
    # Find the index of the last occurrence of "WK8_SUK_V"
    last_index = data.columns.to_list().index('WK8_SUK_V', -2)

    # Change the column name to "WK8_SUK_V_live"
    data.columns.values[last_index] = 'WK8_SUK_1'
    # Find the index of the last occurrence of "WK8_SUK_V"
    last_index = data.columns.to_list().index('WK8_SUK_V', -1)

    # Change the column name to "WK8_SUK_V_live"
    data.columns.values[last_index] = 'WK8_SUK_2'


In [15]:
# This condition happen beacause we dont have week number before it so it doesnt have week number 
if 'WKNone_SUK_V' in data.columns:
    data = data.rename(columns={'WKNone_SUK_V': 'WK0_SUK_V'})


In [16]:
# Cheaking for duplicate data
data.duplicated().sum()

0

In [17]:
# Locating columns for extracting only numbers
col1 = data.iloc[:,1:]

In [18]:
col1.dtypes


WK0_SUK_V      object
WK0_Master1    object
WK1_V1         object
WK1_V2         object
WK1_Master2    object
WK2_V1         object
WK2_V2         object
WK2_V3         object
WK2_V4         object
WK2_Master3    object
WK3_V1         object
WK3_V2         object
WK3_V3         object
WK3_V4         object
WK3_Master4    object
WK4_V1         object
WK4_V2         object
WK4_V3         object
WK4_V4         object
WK4_V5         object
WK4_V6         object
WK4_V7         object
WK4_Master5    object
WK5_V1         object
WK6_SUK_V      object
WK7_V1         object
WK8_SUK_1      object
WK8_SUK_2      object
dtype: object

In [19]:
# Code for extracting only numbers from dataset
for column in [i for i in col1.columns if col1[i].dtype == 'object']:
    data[column] = data[column].astype(str).str.extract('(\d+)').astype(float)

In [20]:
# Filling Null values with zero
fillna = data.iloc[:,1:]
fillnacol=fillna.columns
data[fillnacol]=data[fillnacol].fillna(0)

In [21]:
data.head()

,Email,WK0_SUK_V,WK0_Master1,WK1_V1,WK1_V2,WK1_Master2,WK2_V1,WK2_V2,WK2_V3,WK2_V4,WK2_Master3,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK3_Master4,WK4_V1,WK4_V2,WK4_V3,WK4_V4,WK4_V5,WK4_V6,WK4_V7,WK4_Master5,WK5_V1,WK6_SUK_V,WK7_V1,WK8_SUK_1,WK8_SUK_2
0,vijayacads@gmail.com,18.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151.0,0.0,0.0,0.0,0.0,0.0
1,sreejith.sreenivasan@vigyanshaala.com,2271.0,1701.0,5.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,monikadaw5353@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,prachijagdale0123@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nikitasingh4615@gmail.com,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5192.0,0.0,0.0,0.0,2795.0,0.0


# Enroll Date MySQL Table

In [22]:
# Creating new table which for Enroll_Dates which is taken from incubator graphy sheet
Enroll=pd.DataFrame(df[["Email", 'Enroll Date']])

In [23]:
# Extracting only Enroll_Date
Enroll[['Enroll Date','Time']]=Enroll['Enroll Date'].str.split(' ',expand=True)
Enroll=Enroll.drop(["Time"],axis=1)

In [24]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="Kalpana_Incubator_Sept_Dec_2023_Batch",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [25]:
# Inserting data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("insert IGNORE into 03_enroll_date (Email,Incubator) values(%s,%s)",tuple(row))

In [26]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("REPLACE into 03_enroll_date (Email,incubator) values(%s,%s)",tuple(row))

In [27]:
conn.commit()

# Payment details MySQL table

In [28]:
# Creating new table which for Payment which is taken from incubator graphy sheet
Payment=pd.DataFrame(df[["Email","Assigned Through"]])

In [29]:
# Extracting only Payment removing Order Id
Payment[["Assigned Through","Order_ID"]]=Payment["Assigned Through"].str.split("-",expand=True)
Payment.drop(["Order_ID"],axis=1,inplace=True)

In [30]:
# Assigning fee cost to different cathegory of enrollment
Payment["Assigned Through"]=Payment["Assigned Through"].replace(['Admin'],'0')
Payment["Assigned Through"]=Payment["Assigned Through"].replace(['Excel Upload'],'0')
Payment["Assigned Through"]=Payment["Assigned Through"].replace(['Paid Transaction '],'1899')

In [31]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="Kalpana_Incubator_Sept_Dec_2023_Batch",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [32]:
# Inserting data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("insert Ignore into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [33]:
# Replacing data with new data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("REPLACE into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [34]:
conn.commit()

# Changing seconds to Percentage

In [35]:
# Saving previous column name 
newcol = list(data.columns)


In [36]:
newcol

['Email',
 'WK0_SUK_V',
 'WK0_Master1',
 'WK1_V1',
 'WK1_V2',
 'WK1_Master2',
 'WK2_V1',
 'WK2_V2',
 'WK2_V3',
 'WK2_V4',
 'WK2_Master3',
 'WK3_V1',
 'WK3_V2',
 'WK3_V3',
 'WK3_V4',
 'WK3_Master4',
 'WK4_V1',
 'WK4_V2',
 'WK4_V3',
 'WK4_V4',
 'WK4_V5',
 'WK4_V6',
 'WK4_V7',
 'WK4_Master5',
 'WK5_V1',
 'WK6_SUK_V',
 'WK7_V1',
 'WK8_SUK_1',
 'WK8_SUK_2']

In [37]:
# Saving previous column name for data1 by removing "Email column"
data_columns = list(data.columns)

# Exclude the 'email' column if it exists
if 'Email' in data_columns:
    data_columns.remove('Email')

data1col = data_columns




In [38]:
oldcol = [col for col in oldcol if not col.startswith("Week")]
num_columns_to_rename = min(len(data.columns), len(oldcol))
data.rename(columns=dict(zip(data.columns[:num_columns_to_rename], oldcol[:num_columns_to_rename])), inplace=True)
data.columns = data.columns.str.split (':').str [0]
data.columns = data.columns.str.strip()


In [39]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [40]:
# Rename the column names using the rename_column function
data.rename(columns=rename_column, inplace=True)

In [41]:
data.columns

Index(['Email', 'SUK0', 'MC01', 'VID01', 'VID02', 'MC02', 'VID03', 'VID04',
       'VID05', 'VID06', 'MC03', 'VID07', 'VID08', 'VID09', 'VID10', 'MC04',
       'VID11', 'VID12', 'VID13', 'VID14', 'VID15', 'VID16', 'VID17', 'MC05',
       'VID18', 'SUK1', 'VID20', 'SUK2', 'SUK3'],
      dtype='object')

In [42]:
# Reading Excel file form our source
excel_file  = pd.read_excel(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Source File\00_Source_Kalpana Video Details\Timestamp_of_Videos.xlsx")

In [43]:
data1=pd.DataFrame()

In [44]:
# Iterate over each column in the 'data' DataFrame
for column_name in data.columns:
    # Check if the column exists in the Excel file
    if column_name in excel_file['Code'].values:
        # Get the corresponding value in the "Time" column from the Excel file
        value = excel_file.loc[excel_file['Code'] == column_name, 'Time'].values[0]
        
        # Calculate the percentage value
        percentage_value = (data[column_name] * 100) / value
        
        # Assign the calculated percentage value to a new column in the "data1" DataFrame with the same name as the "Code" column
        data1[column_name] = percentage_value
    else:
        # Handle the case when the column is present in 'data' but not in the Excel file
        print(f"Column '{column_name}' present in 'data' but not in the Excel file. Skipping.")


Column 'Email' present in 'data' but not in the Excel file. Skipping.


In [45]:
# Iterate over each row in the Excel file
for index, row in excel_file.iterrows():
    # Get the value in the "Code" column
    column_name = row['Code']
    
    # Get the value in the "Time" column
    value = row['Time']
    
    # Check if the 'Code' column exists in the 'data' DataFrame
    if column_name in data.columns:
        # Calculate the percentage value
        percentage_value = (data[column_name] * 100) / value
        
        # Assign the calculated percentage value to a new column in the "data1" DataFrame with the same name as the "Code" column
        data1[column_name] = percentage_value

In [46]:
data.head(15)

,Email,SUK0,MC01,VID01,VID02,MC02,VID03,VID04,VID05,VID06,MC03,VID07,VID08,VID09,VID10,MC04,VID11,VID12,VID13,VID14,VID15,VID16,VID17,MC05,VID18,SUK1,VID20,SUK2,SUK3
0,vijayacads@gmail.com,18.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151.0,0.0,0.0,0.0,0.0,0.0
1,sreejith.sreenivasan@vigyanshaala.com,2271.0,1701.0,5.0,0.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,monikadaw5353@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,prachijagdale0123@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nikitasingh4615@gmail.com,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5192.0,0.0,0.0,0.0,2795.0,0.0
5,keerthishivampeta51@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,habibrutuja2003@gmail.com,0.0,470.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,florenceadekojo59@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,sreevishnumohanraj@gmail.com,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,bhargavigpatil64@gmail.com,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Changing column name back to new column
data.columns = newcol 
data1.columns = data1col

In [48]:
data.columns

Index(['Email', 'WK0_SUK_V', 'WK0_Master1', 'WK1_V1', 'WK1_V2', 'WK1_Master2',
       'WK2_V1', 'WK2_V2', 'WK2_V3', 'WK2_V4', 'WK2_Master3', 'WK3_V1',
       'WK3_V2', 'WK3_V3', 'WK3_V4', 'WK3_Master4', 'WK4_V1', 'WK4_V2',
       'WK4_V3', 'WK4_V4', 'WK4_V5', 'WK4_V6', 'WK4_V7', 'WK4_Master5',
       'WK5_V1', 'WK6_SUK_V', 'WK7_V1', 'WK8_SUK_1', 'WK8_SUK_2'],
      dtype='object')

In [49]:
data1.columns

Index(['WK0_SUK_V', 'WK0_Master1', 'WK1_V1', 'WK1_V2', 'WK1_Master2', 'WK2_V1',
       'WK2_V2', 'WK2_V3', 'WK2_V4', 'WK2_Master3', 'WK3_V1', 'WK3_V2',
       'WK3_V3', 'WK3_V4', 'WK3_Master4', 'WK4_V1', 'WK4_V2', 'WK4_V3',
       'WK4_V4', 'WK4_V5', 'WK4_V6', 'WK4_V7', 'WK4_Master5', 'WK5_V1',
       'WK6_SUK_V', 'WK7_V1', 'WK8_SUK_1', 'WK8_SUK_2'],
      dtype='object')

In [50]:
# Creating a function "Max_Value" for converting % max 100 if values are above 100
def Max_Value(value):
    if value >=100:
        return 100
    else:
        return value

In [51]:
# Applying Max_Value function
per1=data1.iloc[:,0:]
per1c=per1.columns
data1[per1c]=data1[per1c].applymap(Max_Value)

# Code for converting seconds to hours

In [52]:
# Creating a function for converting seconds to hours
def Convert_Hours(seconds):
    hours = seconds / (3600)
    return hours
n = 5400
print(Convert_Hours(n))


1.5


In [53]:
# Applying Convert_Hours function
col1 = data.iloc[:,1:]
col=col1.columns
data[col]=data[col].apply(Convert_Hours)

# Code for Recorded Videos: Total hours, Average & Percentage

## Input Weeks Here ↓ 

In [54]:
# Define the week number up to which you want to include columns
end_week = 8

In [55]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_V{video}'
        if col_name in data.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Recorded_Total = data[cols_to_select]

In [56]:
Recorded_Total.head()

,WK1_V1,WK1_V2,WK2_V1,WK2_V2,WK2_V3,WK2_V4,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK4_V1,WK4_V2,WK4_V3,WK4_V4,WK4_V5,WK4_V6,WK4_V7,WK5_V1,WK7_V1
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.001389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
data["Recorded_Total"]=Recorded_Total.sum(axis=1)

In [58]:
data["Recorded_Average"]=Recorded_Total.mean(axis=1)

In [59]:
# Create a list of all column names up to the end week
cols_to_select_per = []
for week in range(end_week + 1):
    for video in range(1, 10):
        col_name = f'WK{week}_V{video}'
        if col_name in data1.columns:
            cols_to_select_per.append(col_name)

# Select the desired columns and create a new DataFrame
Recorded_Percent = data1[cols_to_select_per]

In [60]:
Recorded_Percent.head()

,WK1_V1,WK1_V2,WK2_V1,WK2_V2,WK2_V3,WK2_V4,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK4_V1,WK4_V2,WK4_V3,WK4_V4,WK4_V5,WK4_V6,WK4_V7,WK5_V1,WK7_V1
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.718213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
data["Recorded_Percentage"]=Recorded_Percent.mean(axis=1)

# Code for SUK Recorded Videos: Total hours, Average & Percentage

In [62]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in data.columns:
        cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
SUK_Recorded_Total = data[cols_to_select]

In [63]:
SUK_Recorded_Total.head()

,WK0_SUK_V,WK6_SUK_V
0,0.005000,0.0
1,0.630833,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.000000,0.0


In [64]:
data["SUK_Recorded_Total"]=SUK_Recorded_Total.sum(axis=1)

In [65]:
data["SUK_Recorded_Average"]=SUK_Recorded_Total.mean(axis=1)

In [66]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in data1.columns:
        cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
SUK_Recorded_Percent = data1[cols_to_select]

In [67]:
data["SUK_Recorded_Percentage"]=SUK_Recorded_Percent.mean(axis=1)

# Code for Master Class: Total hours, Average & Percentage

In [68]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 2):
    for masterclass in range(1, 10):
        col_name = f'WK{week}_Master{masterclass}'
        if col_name in data.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Masterclass_Total = data[cols_to_select]


In [69]:
data["Masterclass_Total"]=Masterclass_Total.sum(axis=1)

In [70]:
data["Masterclass_Average"]=Masterclass_Total.mean(axis=1)

In [71]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    for masterclass in range(1, 10):
        col_name = f'WK{week}_Master{masterclass}'
        if col_name in data1.columns:
            cols_to_select.append(col_name)

# Select the desired columns and create a new DataFrame
Masterclass_Percent = data1[cols_to_select]


In [72]:
Masterclass_Percent.head()

,WK0_Master1,WK1_Master2,WK2_Master3,WK3_Master4,WK4_Master5
0,0.277778,0.0,0.0,0.0,4.194444
1,47.250000,2.5,0.0,0.0,0.000000
2,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.000000
4,0.111111,0.0,0.0,0.0,100.000000


In [73]:
data["Masterclass_Percentage"]=Masterclass_Percent.mean(axis=1)

# Code for Program: Total hours, Average & Percentage

In [74]:
# select columns from data where the column names start with "WK"
selected_cols = [col for col in data.columns if col.startswith('WK')]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Total = data[selected_cols]

In [75]:
Whole_Program_Total.head()

,WK0_SUK_V,WK0_Master1,WK1_V1,WK1_V2,WK1_Master2,WK2_V1,WK2_V2,WK2_V3,WK2_V4,WK2_Master3,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK3_Master4,WK4_V1,WK4_V2,WK4_V3,WK4_V4,WK4_V5,WK4_V6,WK4_V7,WK4_Master5,WK5_V1,WK6_SUK_V,WK7_V1,WK8_SUK_1,WK8_SUK_2
0,0.005000,0.002778,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041944,0.0,0.0,0.0,0.000000,0.0
1,0.630833,0.472500,0.001389,0.0,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,0.000000,0.001111,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.442222,0.0,0.0,0.0,0.776389,0.0


In [76]:
data["Program_Total"]=Whole_Program_Total.sum(axis=1)

In [77]:
data["Program_Average"]=Whole_Program_Total.mean(axis=1)

In [78]:
# select columns from data where the column names start with "WK"
selected_cols = [col for col in data1.columns if col.startswith('WK')]

# create a new dataframe called Whole_Program_Total with the selected columns
Whole_Program_Percent = data1[selected_cols]

In [79]:
data["Program_percentage"]=Whole_Program_Percent.mean(axis=1)

In [80]:
data=data.round(2)

In [81]:
# Saving In Database
data.to_csv(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Output\Incubator_and_attendence_monitoring_Sep_2023.csv",mode='w',index=False)

# Incubator and attendance monitoring MySQL table

In [82]:
data.columns

Index(['Email', 'WK0_SUK_V', 'WK0_Master1', 'WK1_V1', 'WK1_V2', 'WK1_Master2',
       'WK2_V1', 'WK2_V2', 'WK2_V3', 'WK2_V4', 'WK2_Master3', 'WK3_V1',
       'WK3_V2', 'WK3_V3', 'WK3_V4', 'WK3_Master4', 'WK4_V1', 'WK4_V2',
       'WK4_V3', 'WK4_V4', 'WK4_V5', 'WK4_V6', 'WK4_V7', 'WK4_Master5',
       'WK5_V1', 'WK6_SUK_V', 'WK7_V1', 'WK8_SUK_1', 'WK8_SUK_2',
       'Recorded_Total', 'Recorded_Average', 'Recorded_Percentage',
       'SUK_Recorded_Total', 'SUK_Recorded_Average', 'SUK_Recorded_Percentage',
       'Masterclass_Total', 'Masterclass_Average', 'Masterclass_Percentage',
       'Program_Total', 'Program_Average', 'Program_percentage'],
      dtype='object')

In [83]:
data.head()

,Email,WK0_SUK_V,WK0_Master1,WK1_V1,WK1_V2,WK1_Master2,WK2_V1,WK2_V2,WK2_V3,WK2_V4,WK2_Master3,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK3_Master4,WK4_V1,WK4_V2,WK4_V3,WK4_V4,WK4_V5,WK4_V6,WK4_V7,WK4_Master5,WK5_V1,WK6_SUK_V,WK7_V1,WK8_SUK_1,WK8_SUK_2,Recorded_Total,Recorded_Average,Recorded_Percentage,SUK_Recorded_Total,SUK_Recorded_Average,SUK_Recorded_Percentage,Masterclass_Total,Masterclass_Average,Masterclass_Percentage,Program_Total,Program_Average,Program_percentage
0,vijayacads@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.25,0.04,0.01,0.89,0.05,0.00,0.18
1,sreejith.sreenivasan@vigyanshaala.com,0.63,0.47,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.09,0.63,0.32,31.54,0.50,0.10,9.95,1.13,0.04,4.09
2,monikadaw5353@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,prachijagdale0123@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,nikitasingh4615@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.44,0.0,0.0,0.0,0.78,0.0,0.0,0.0,0.00,0.00,0.00,0.00,1.44,0.29,20.02,2.22,0.08,6.35


In [84]:
data.columns

Index(['Email', 'WK0_SUK_V', 'WK0_Master1', 'WK1_V1', 'WK1_V2', 'WK1_Master2',
       'WK2_V1', 'WK2_V2', 'WK2_V3', 'WK2_V4', 'WK2_Master3', 'WK3_V1',
       'WK3_V2', 'WK3_V3', 'WK3_V4', 'WK3_Master4', 'WK4_V1', 'WK4_V2',
       'WK4_V3', 'WK4_V4', 'WK4_V5', 'WK4_V6', 'WK4_V7', 'WK4_Master5',
       'WK5_V1', 'WK6_SUK_V', 'WK7_V1', 'WK8_SUK_1', 'WK8_SUK_2',
       'Recorded_Total', 'Recorded_Average', 'Recorded_Percentage',
       'SUK_Recorded_Total', 'SUK_Recorded_Average', 'SUK_Recorded_Percentage',
       'Masterclass_Total', 'Masterclass_Average', 'Masterclass_Percentage',
       'Program_Total', 'Program_Average', 'Program_percentage'],
      dtype='object')

In [85]:
oldcol

['Email',
 'SUK + : Kalpana Inauguration, at 6:30 PM. on Saturday, 30th Sept 2023 - 65086523e4b0a106c0f6312b',
 'Master Class ! : Career Exploration - Saturday, Oct 7, 6:30 PM - 652153e8e4b09e4247b99db1',
 'Video ! : Why women in STEM. What has Kalpana program achieved until now. - 62c7d78d0cf273488220b8d4',
 'Video @ : What we expect from our Kalpana fellows - 62c7d10e0cf273488220b5bf',
 'Master Class @ : Planning your Applications – Higher Education, Saturday, Oct 14, 6:30 PM - 650870f7e4b055d84f1698b5',
 'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
 'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
 'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
 'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
 'Master Class # : Searching and Securing Internships - Saturday, Oct 21, 6:30 PM - 650873c4e4b0412df5285380',
 'Video & : What is SWOT & its importance - 62b2a68c0cf2aa41107c5324',
 'Video * : How to d

In [86]:
oldcol = [col for col in oldcol if not col.startswith("Week")]


In [87]:
oldcol

['Email',
 'SUK + : Kalpana Inauguration, at 6:30 PM. on Saturday, 30th Sept 2023 - 65086523e4b0a106c0f6312b',
 'Master Class ! : Career Exploration - Saturday, Oct 7, 6:30 PM - 652153e8e4b09e4247b99db1',
 'Video ! : Why women in STEM. What has Kalpana program achieved until now. - 62c7d78d0cf273488220b8d4',
 'Video @ : What we expect from our Kalpana fellows - 62c7d10e0cf273488220b5bf',
 'Master Class @ : Planning your Applications – Higher Education, Saturday, Oct 14, 6:30 PM - 650870f7e4b055d84f1698b5',
 'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
 'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
 'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
 'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
 'Master Class # : Searching and Securing Internships - Saturday, Oct 21, 6:30 PM - 650873c4e4b0412df5285380',
 'Video & : What is SWOT & its importance - 62b2a68c0cf2aa41107c5324',
 'Video * : How to d

In [88]:
#data.columns = oldcol[:len(data.columns)]
# Now, let's say you want to change the column names of 'old_dataset' based on 'filtered_column_names'
num_columns_to_rename = min(len(data.columns), len(oldcol))
print(num_columns_to_rename)

29


In [89]:
data.rename(columns=dict(zip(data.columns[:num_columns_to_rename], oldcol[:num_columns_to_rename])), inplace=True)


In [90]:
data.columns

Index(['Email',
       'SUK + : Kalpana Inauguration, at 6:30 PM. on Saturday, 30th Sept 2023 - 65086523e4b0a106c0f6312b',
       'Master Class ! : Career Exploration - Saturday, Oct 7, 6:30 PM - 652153e8e4b09e4247b99db1',
       'Video ! : Why women in STEM. What has Kalpana program achieved until now. - 62c7d78d0cf273488220b8d4',
       'Video @ : What we expect from our Kalpana fellows - 62c7d10e0cf273488220b5bf',
       'Master Class @ : Planning your Applications – Higher Education, Saturday, Oct 14, 6:30 PM - 650870f7e4b055d84f1698b5',
       'Video # : Goal setting - Part 1 - 6529ffc5e4b0a564951cb119',
       'Video $ : Goal setting - Part 2 - 6529ffd4e4b08a0cfb8d0628',
       'Video % : What is SMART goal ? - 62b3fded0cf2f5120a60c932',
       'Video ^ : What is the difference between a goal and a SMART goal ? - 62b3fe320cf2f5120a60c946',
       'Master Class # : Searching and Securing Internships - Saturday, Oct 21, 6:30 PM - 650873c4e4b0412df5285380',
       'Video & : What is

In [91]:
# Split the columns by ':' and keep only the first part
data.columns = data.columns.str.split (':').str [0]

In [92]:
data.columns

Index(['Email', 'SUK + ', 'Master Class ! ', 'Video ! ', 'Video @ ',
       'Master Class @ ', 'Video # ', 'Video $ ', 'Video % ', 'Video ^ ',
       'Master Class # ', 'Video & ', 'Video * ', 'Video ? ', 'Video !+ ',
       'Master Class $ ', 'Video !! ', 'Video !@ ', 'Video !# ', 'Video !$ ',
       'Video !% ', 'Video !^ ', 'Video !& ', 'Master Class % ', 'Video !* ',
       'SUK ! ', 'Video @+ ', 'SUK @ ', 'SUK #', 'Recorded_Total',
       'Recorded_Average', 'Recorded_Percentage', 'SUK_Recorded_Total',
       'SUK_Recorded_Average', 'SUK_Recorded_Percentage', 'Masterclass_Total',
       'Masterclass_Average', 'Masterclass_Percentage', 'Program_Total',
       'Program_Average', 'Program_percentage'],
      dtype='object')

In [93]:
data.columns = data.columns.str.strip()


In [94]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col


In [95]:
# Rename the column names using the rename_column function
data.rename(columns=rename_column, inplace=True)

In [96]:
data.columns

Index(['Email', 'SUK0', 'MC01', 'VID01', 'VID02', 'MC02', 'VID03', 'VID04',
       'VID05', 'VID06', 'MC03', 'VID07', 'VID08', 'VID09', 'VID10', 'MC04',
       'VID11', 'VID12', 'VID13', 'VID14', 'VID15', 'VID16', 'VID17', 'MC05',
       'VID18', 'SUK1', 'VID20', 'SUK2', 'SUK3', 'Recorded_Total',
       'Recorded_Average', 'Recorded_Percentage', 'SUK_Recorded_Total',
       'SUK_Recorded_Average', 'SUK_Recorded_Percentage', 'Masterclass_Total',
       'Masterclass_Average', 'Masterclass_Percentage', 'Program_Total',
       'Program_Average', 'Program_percentage'],
      dtype='object')

In [97]:
data.head()

,Email,SUK0,MC01,VID01,VID02,MC02,VID03,VID04,VID05,VID06,MC03,VID07,VID08,VID09,VID10,MC04,VID11,VID12,VID13,VID14,VID15,VID16,VID17,MC05,VID18,SUK1,VID20,SUK2,SUK3,Recorded_Total,Recorded_Average,Recorded_Percentage,SUK_Recorded_Total,SUK_Recorded_Average,SUK_Recorded_Percentage,Masterclass_Total,Masterclass_Average,Masterclass_Percentage,Program_Total,Program_Average,Program_percentage
0,vijayacads@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.25,0.04,0.01,0.89,0.05,0.00,0.18
1,sreejith.sreenivasan@vigyanshaala.com,0.63,0.47,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.09,0.63,0.32,31.54,0.50,0.10,9.95,1.13,0.04,4.09
2,monikadaw5353@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,prachijagdale0123@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,nikitasingh4615@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.44,0.0,0.0,0.0,0.78,0.0,0.0,0.0,0.00,0.00,0.00,0.00,1.44,0.29,20.02,2.22,0.08,6.35


In [98]:
mask = data.columns.duplicated().sum()
print(mask)

0


# Storing data on MySQL

In [99]:
merged = df.merge(data, on ='Email', how = 'inner') 

In [100]:
merged = merged.drop('Enroll Date', axis=1)

In [101]:
merged = merged.rename(columns={'Assigned Through': 'Assigned_Through'})
merged = merged.rename(columns={'Mobile': 'Phone'})
merged = merged.rename(columns={'Course Name': 'Course_Name'})

In [102]:
merged.head()

,Name,Email,Segment,Phone,Assigned_Through,Course_Name,SUK0,MC01,VID01,VID02,MC02,VID03,VID04,VID05,VID06,MC03,VID07,VID08,VID09,VID10,MC04,VID11,VID12,VID13,VID14,VID15,VID16,VID17,MC05,VID18,SUK1,VID20,SUK2,SUK3,Recorded_Total,Recorded_Average,Recorded_Percentage,SUK_Recorded_Total,SUK_Recorded_Average,SUK_Recorded_Percentage,Masterclass_Total,Masterclass_Average,Masterclass_Percentage,Program_Total,Program_Average,Program_percentage
0,Vijay Venugopalan,vijayacads@gmail.com,trial,9.198907e+11,Admin,Kalpana Incubator Sept- Dec 2023 Batch,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.25,0.04,0.01,0.89,0.05,0.00,0.18
1,Sreejith Fourth,sreejith.sreenivasan@vigyanshaala.com,free,9.322869e+09,Admin,Kalpana Incubator Sept- Dec 2023 Batch,0.63,0.47,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.09,0.63,0.32,31.54,0.50,0.10,9.95,1.13,0.04,4.09
2,Aparna Daw,monikadaw5353@gmail.com,free,9.181790e+11,Excel Upload,Kalpana Incubator Sept- Dec 2023 Batch,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Prachi Arvind Jagdale,prachijagdale0123@gmail.com,free,8.421484e+09,Excel Upload,Kalpana Incubator Sept- Dec 2023 Batch,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,NIKITA SINGH,nikitasingh4615@gmail.com,free,9.185640e+11,Excel Upload,Kalpana Incubator Sept- Dec 2023 Batch,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.44,0.0,0.0,0.0,0.78,0.0,0.0,0.0,0.00,0.00,0.00,0.00,1.44,0.29,20.02,2.22,0.08,6.35


In [103]:
conn= msql.connect(host='localhost',user='root',password="VS@123",database="kalpana",auth_plugin='mysql_native_password')
cursor =conn.cursor() 

In [104]:
# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM 08_incubator_and_attendance_monitoring")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Recorded_Total'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in merged.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "INT" 
            alter_query = f"ALTER TABLE 08_incubator_and_attendence_monitoring ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index - 1]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)

In [105]:
# Your existing code for inserting data into the database table
for i, row in merged.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(merged.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 08_incubator_and_attendance_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in merged.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [106]:
conn.commit()

# Extracting desire output to excel sheet

In [107]:
data.head()

,Email,SUK0,MC01,VID01,VID02,MC02,VID03,VID04,VID05,VID06,MC03,VID07,VID08,VID09,VID10,MC04,VID11,VID12,VID13,VID14,VID15,VID16,VID17,MC05,VID18,SUK1,VID20,SUK2,SUK3,Recorded_Total,Recorded_Average,Recorded_Percentage,SUK_Recorded_Total,SUK_Recorded_Average,SUK_Recorded_Percentage,Masterclass_Total,Masterclass_Average,Masterclass_Percentage,Program_Total,Program_Average,Program_percentage
0,vijayacads@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.25,0.04,0.01,0.89,0.05,0.00,0.18
1,sreejith.sreenivasan@vigyanshaala.com,0.63,0.47,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.09,0.63,0.32,31.54,0.50,0.10,9.95,1.13,0.04,4.09
2,monikadaw5353@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,prachijagdale0123@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,nikitasingh4615@gmail.com,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.44,0.0,0.0,0.0,0.78,0.0,0.0,0.0,0.00,0.00,0.00,0.00,1.44,0.29,20.02,2.22,0.08,6.35
